In [ ]:
# With retrieval augmented generation, we use the following steps to generate context aware answers:

#     Convert the prompt (question text) into embedding.
#     (R) Retrieve N most relevant entries from the knowledge base. This is treated as the context of the conversation.
#     (A) Augment the prompt with the context by prepending the context to the question text. This end result is the context aware prompt.
#     (G)" Generate the answer by feeding the context aware prompt to the foundation model.

In [3]:
region = 'us-west-2'
host = 'https://b19fa6b2rsx72rc9osh8.us-west-2.aoss.amazonaws.com'


In [7]:
import boto3
import json
import requests
from requests_aws4auth import AWS4Auth

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding
    
def search(embedding, limit=1):
    # prepare for OpenSearch Serverless
    service = 'aoss'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(
        credentials.access_key, 
        credentials.secret_key, 
        region, 
        service, 
        session_token=credentials.token
    )
    # search
    index = 'demo-index'
    datatype = '_search'
    url = host + '/' + index + '/' + datatype
    headers = {'Content-Type': 'application/json'}
    document = {
        'size': limit,
        'query': {
            'knn': {
                'embedding': {
                    'vector': embedding,
                    'k': limit
                }
            }
        }
    }
    # response
    response = requests.get(url, auth=awsauth, json=document, headers=headers)
    response.raise_for_status()
    data = response.json()
    output = ''
    for item in data['hits']['hits']:
        output += item['_source']['content'] + '\n'
    return output

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
# this is the original prompt (query text)
prompt = 'What does Albert Einstein do?'
# convet the query text into embedding
embedding = get_embedding(bedrock, prompt)
# retrieve 5 most relevant entries from the knowledge base
info = search(embedding, limit=5)
# augment the prompt with the context
RAGprompt = 'Use the context below to answer the question:\n\n=== Context ===\n{0}\n\n=== Question ===\n{1}'.format(info, prompt)
# ask the foundation model
modelId = 'ai21.j2-ultra'
accept = 'application/json'
contentType = 'application/json'
input = {'prompt': prompt, 'maxTokens': 200}
input1 = {'prompt': RAGprompt, 'maxTokens': 200}
body=json.dumps(input)
response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept,contentType=contentType)
response_body = json.loads(response.get('body').read())
completions = response_body['completions']
for part in completions:
    print(part['data']['text'])

body1=json.dumps(input1)
response = bedrock.invoke_model(body=body1, modelId=modelId, accept=accept,contentType=contentType)
response_body = json.loads(response.get('body').read())
completions = response_body['completions']
for part in completions:
    print(part['data']['text'])



Albert Einstein worked as a patent clerk shortly after his marriage to Mileva Marić. In 1909 he obtained a associate professorship at the University of Bern and in 1912 became an Professor of Theoretical Physics at the University of Zurich. Eventually Einstein moved to Berlin in 1914 where he became a member of the Prussian Academy of Sciences and a professor at the Humboldt University of Berlin. From onward Einstein was spends the rest of his his time working in theoretical physics from 1914 onwards.  This was where he had his most important discoveries which gave him his fame. In 1921, he was awarded the Nobel Prize in Physics "for his services to Theoretical Physics, and especially for his discovery of the law of the photoelectric effect". In the late 1920s, Einstein faced terrible trials of cancers of the economy which lead him to move to America in 1933 where be lectured at several universities. Throughout the rest of his life Einstein continued to work on physics and other vario